В предишната тетрадка направихме речник от последователности от символи. Нещо което не отбелязахме, но личеше в данните, е че някои последователности се появяваха в различни варианти на големи/малки букви. Например - `the`/`The`. Ще се опитаме да разерем колко често се случва и дали си заслужава споециална обработка.

In [1]:
def articles():
    with open('page_revisions_text', 'rb') as text_file:
        pending_article_data = b''
        while True:
            data = text_file.read(1024 * 1024)
            if len(data) == 0:
                break

            articles = data.split(b'\0')
            articles[0] = pending_article_data + articles[0]
            for index, article in enumerate(articles):
                if index + 1 == len(articles):
                    pending_article_data = article
                else:
                    yield article

        if len(pending_article_data) != 0:
            yield pending_article_data

In [2]:
import re

word_matcher = re.compile('\w+', re.U)

In [17]:
import collections

words = collections.defaultdict(lambda: 0)

for article in articles():
    for match in word_matcher.finditer(article.decode('utf-8')):
        words[match.group()] += 1

Нека видим малко статистика. Колко думи имаме и коя е най-дългата дума...

In [18]:
len(words)

1990305

In [32]:
max(len(word) for word in words.keys())

880

Това е изненадващо. Да погледнем по-отблизо най-дългите думи...

In [36]:
words_by_length = sorted(words.keys(), key=len, reverse=True)

In [37]:
words_by_length

['40236_rQ3D1Q26title1Q3DQ26title2Q3DRANQ2520Q2528MOVIEQ2529Q26reviewerQ3DVincentQ2520CanbyQ26pdateQ3D19860622Q26v_idQ3D40236_rQ513D1Q5126title1Q513DQ5126title2Q513DRANQ512520Q512528MOVIEQ512529Q5126reviewerQ513DVincentQ512520CanbyQ5126pdateQ513D19860622Q5126v_idQ513D40236_rQ51513D1Q515126title1Q51513DQ515126title2Q51513DRANQ51512520Q51512528MOVIEQ51512529Q515126reviewerQ51513DVincentQ51512520CanbyQ515126pdateQ51513D19860622Q515126v_idQ51513D40236_rQ5151513D1Q51515126title1Q5151513DQ51515126title2Q5151513DRANQ5151512520Q5151512528MOVIEQ5151512529Q51515126reviewerQ5151513DVincentQ5151512520CanbyQ51515126pdateQ5151513D19860622Q51515126v_idQ5151513D40236_rQ515151513D1Q5151515126title1Q515151513DQ5151515126title2Q515151513DRANQ515151512520Q515151512528MOVIEQ515151512529Q5151515126reviewerQ515151513DVincentQ515151512520CanbyQ5151515126pdateQ515151513D19860622Q5151515126v_idQ',
 '11111111111111111111111111111111111111111111111111111112222222222222222222222222222222222222222222222222222222222

Има странни неща. Няколко очевидни инстанции на Wikipedia вандализъм. И няколко на шимпанзета с пишещи машини и (безкрайно) много време. Също така има доста числа в двоично, десетично, шестнайсетично и base64 представяне. Прави впечатление и ползването на долна черта за разделител. Ползвайки `\w`, регулярният ни израз не се справя добре с това.

Ще трябва да живеем с повечето от тези. Но нека поне обработим долните черити правилно...

In [48]:
words = collections.defaultdict(lambda: 0)

for article in articles():
    for match in word_matcher.finditer(article.decode('utf-8')):
        for word in match.group().strip('_').split('_'):
            words[word] += 1

In [49]:
len(words)

1889077

Вече са с около 100 000 по-малко. От чисто любопитство - нека видим отново най-дългите...

In [50]:
words_by_length = sorted(words.keys(), key=len, reverse=True)

In [51]:
words_by_length

['11111111111111111111111111111111111111111111111111111112222222222222222222222222222222222222222222222222222222222222222222222222233333333333333333333333333333333333333333333333333333333333334444444444444444444444444444444444444444444444444444444444455555555555555555555555555555555555555555555555566666666666666666666666666677777777777777777777777777777777777777777777777777778888888888888888888888888888888888888888888888888888999999999999999999999999999999999999999999999990000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 'moooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

Нека се върнем към оригиналния план. Да разгледаме различното изписване в малки/големи букви на думите.

In [57]:
lowcase_words = collections.defaultdict(lambda: set())

for word in words:
    lowcase_words[word.lower()].add(word)

In [59]:
len(lowcase_words)

1637016

In [60]:
len(words) - len(lowcase_words)

252061

In [61]:
sorted(lowcase_words.values(), key=len, reverse=True)

[{'BWBwbwbwb',
  'BWbwBwbwb',
  'BWbwbwBwb',
  'BWbwbwbwB',
  'BwBWbwbwb',
  'BwBwbWbwb',
  'BwBwbwbWb',
  'BwbWBwbwb',
  'BwbWbwBwb',
  'BwbWbwbwB',
  'BwbwBWbwb',
  'BwbwBwbWb',
  'BwbwbWBwb',
  'BwbwbWbwB',
  'BwbwbwBWb',
  'BwbwbwbWB',
  'bWBwBwbwb',
  'bWBwbwBwb',
  'bWBwbwbwB',
  'bWbWbWbwb',
  'bWbWbwbWb',
  'bWbwBwbwB',
  'bWbwbWbWb',
  'bWbwbwBwB',
  'bwBWBwbwb',
  'bwBWbwBwb',
  'bwBWbwbwB',
  'bwBwBWbwb',
  'bwBwBwbWb',
  'bwBwbWBwb',
  'bwBwbWbwB',
  'bwBwbwBWb',
  'bwBwbwbWB',
  'bwbWBwBwb',
  'bwbWBwbwB',
  'bwbWbWbWb',
  'bwbWbwBwB',
  'bwbwBWBwb',
  'bwbwBWbwB',
  'bwbwBwBWb',
  'bwbwBwbWB',
  'bwbwbWBwB',
  'bwbwbwBWB'},
 {'REDIRECT',
  'REDIRECt',
  'REDIRect',
  'REDIrECT',
  'REDiRECT',
  'REDireCt',
  'REDirect',
  'REdirect',
  'ReDIRECT',
  'Redirect',
  'redIRect',
  'redirect'},
 {'CATEGORY',
  'CAtegory',
  'CategoRY',
  'CategorY',
  'Category',
  'cAtegory',
  'categorY',
  'category'},
 {'SEX', 'SEx', 'SeX', 'Sex', 'sEX', 'sEx', 'seX', 'sex'},
 {'THE', 'THe

In [73]:
letters = set()

for word in words:
    letters |= set(word)

len(letters)

9709

In [75]:
lowcase_letters = set()

for word in lowcase_words:
    lowcase_letters |= set(word)

len(lowcase_letters)

9245